In [ ]:
import sys

# sys.path.append("/home/rajeeva/Project/yeast_growth_pred/")
sys.path.append("/home/rajeeva/Project/yeast_growth_pred/main_code/FGR/")
from functools import partial

import lightning as L
import torch
import torchmetrics

# from rootutils import setup_root
# setup_root("../", indicator=".project-root", pythonpath=True)


# torch.set_grad_enabled(True)
torch.set_float32_matmul_precision("high")

In [ ]:
%cd ../

In [ ]:
from finetuning_scheduler import FinetuningScheduler
from lightning import Trainer

from main_code.data import CancerKFoldModule
from main_code.FGR.load_FGR import get_fgr_model
from main_code.lightning_model import PhenoPredict
from main_code.network import FGRNet

In [ ]:
dm = CancerKFoldModule(
    path="/home/rajeeva/Project/data/cancer/PRISM_19Q4_lung/reg/",
    split_seed=42,
    num_splits=2,
    batch_size=64,
    num_workers=4,
    test_size=0.2,
    fgr=True,
    latent_path="/home/rajeeva/Project/data/cancer/ChemGroup.parquet",
)
dm.setup(stage="fit")

In [ ]:
# Model
fgr_model = get_fgr_model("/home/rajeeva/Project/outputs/fgr_model_pretrain.pt")
# fgr_model.to("cuda:0")
# fgr_model

In [ ]:
net = FGRNet(
    fgr_model=fgr_model,
    hidden_layers=[2048, 1024, 512, 256],
    geno_size=18748,
    latent_size=256,
    output_size=1,
    dropout=0.05,
    activation="relu",
)

In [ ]:
lm = PhenoPredict(
    net=net,
    optimizer=partial(
        torch.optim.AdamW, lr=1e-3, weight_decay=0.01
    ),  # torch.optim.AdamW(lr=1e-3, weight_decay=0.01),
    scheduler=partial(
        torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
        T_0=10,
    ),  # torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(T_max=10),
    loss_function=torch.nn.MSELoss(),
    main_metric=torchmetrics.regression.MeanSquaredError(),
    additional_metrics=torchmetrics.MetricCollection(torchmetrics.regression.MeanAbsoluteError()),
    task="classification",
    compile=False,
)
# lm

In [ ]:
fts_callback = FinetuningScheduler(ft_schedule="./configs/ft_schedule.yaml")
trainer = Trainer(
    accelerator="gpu", max_epochs=10, callbacks=[fts_callback], enable_checkpointing=False
)

In [ ]:
trainer.fit(lm, dm)

In [ ]:
for param in lm.parameters():
    print(param.requires_grad)

In [ ]:
lm.dtype

In [ ]:
for X, y in dm.train_dataloader():
    print(X[0].requires_grad)
    print(X[1].requires_grad)
    print(y.requires_grad)
    break

In [ ]:
lm = lm.to("cpu")

In [ ]:
lm.model_step([X, y])[0].requires_grad

In [ ]:
y.device